In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from PWRTestIO import *
import Keysight34972A
import InstekPSW
import pyvisa
import instrument_strings
import random
import time
import matplotlib.pyplot as plt

In [ ]:
rm = pyvisa.ResourceManager()
rm.list_resources()

In [ ]:
#rm = pyvisa.ResourceManager()
for resource_id in rm.list_resources():
    try:
        inst = rm.open_resource(resource_id, send_end=True) #the VISA resource
        name_str = inst.query('*IDN?').strip()
        if name_str == instrument_strings.Keysight34972A:
            daq = Keysight34972A.Keysight34972A(inst)
            print("Connected to: " + daq.name.rstrip('\n'))
        if name_str == instrument_strings.PSW800:
            hv_supply = InstekPSW.InstekPSW(inst)
            print("Connected to: " + hv_supply.name.rstrip('\n'))
        if name_str == instrument_strings.PSW80V:
            lv_supply = InstekPSW.InstekPSW(inst)
            print("Connected to: " + lv_supply.name.rstrip('\n'))
            
    except pyvisa.errors.VisaIOError:
        print(resource_id + " is not what we're looking for, continuing...\n")


In [ ]:
lv_supply.set_output(0)
load1_off(daq)
load2_off(daq)

close_rl5(daq) #power PSUa, 


lv_supply.apply(24,2)
lv_supply.set_output(1)


In [ ]:
for i in range(10):
    buck_on(daq)
    flyback_on(daq)
    #open_rl5(daq)
    time.sleep(1)
    buck_off(daq)
    flyback_off(daq)
    #close_rl5(daq)
    time.sleep(1)
#lv_supply.set_output(0)
print('done')

In [ ]:
lv_supply.set_output(0)

In [ ]:
set_discharge_Vgs(daq, 0)
def measure_HVCap(daq):
    return daq.measure_DCV(210)*4
def measure_buckCurrent(daq):
    v_rsns = daq.measure_DCV(215)
    return v_rsns/.28


In [ ]:

flyback_off(daq)

In [ ]:

set_discharge_Vgs(daq, 0)
time.sleep(1)
print(daq.measure_Resistance(210))

In [ ]:
set_discharge_Vgs(daq, 10)
time.sleep(1)
print(daq.measure_Resistance(210))

In [ ]:
for i in range(4):
    voltage = measure_HVCap(daq)
    current = measure_buckCurrent(daq)
    print('voltage: {}'.format(voltage))
    print('current: {}'.format(current))
    print('power: {}'.format(voltage*current))
    time.sleep(.5)

In [ ]:
def set_discharge_Vgs(daq, voltage):
    daq.analog_source(104, voltage)
voltages = []
currents = []
powers = []
def discharge_caps(vgs, voltage, current, daq):
    set_discharge_Vgs(daq, vgs)
    voltages.append(voltage)
    currents.append(currents)
    powers.append(voltage*current)
    if voltage < 1 and current < .1:
        set_discharge_Vgs(daq, 10)
        time.sleep(5)
        print('caps should be discharged')
        set_discharge_Vgs(daq, 0)
        return 
    else:
        power = voltage*current
        if power > 100:
            discharge_caps(max(0,vgs-.1),measure_HVCap(daq),measure_buckCurrent(daq),daq )
        elif power < 60:
            discharge_caps(min(vgs+.1,10),measure_HVCap(daq),measure_buckCurrent(daq),daq )
        else:
            discharge_caps(vgs, measure_HVCap(daq),measure_buckCurrent(daq),daq )

In [ ]:
discharge_caps(1, 4, 1,daq)

In [ ]:
df = pd.DataFrame({'voltages':[1,2,4],'currents':[3,4,1],'powers':[7,8,9]})
plt.figure()
df.plot()